In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 10.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78935 sha256=8889dfc1f65ad3ccff437114c41e532bde2554656d5fc8fe49bee13ac32c6726
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=3deb261bff2c064df4dfae5df260359830ad37f808722879da1191bef0af0ebf
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.5.0


In [4]:
fashion_mnsit = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnsit.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3, 5]),
        activation='relu',
        input_shape=(28, 28, 1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model


In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=5,
                          directory='output', project_name='Mnist Fashion')

In [11]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.9100000262260437

Best val_accuracy So Far: 0.9100000262260437
Total elapsed time: 00h 02m 42s
INFO:tensorflow:Oracle triggered exit


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,348,906
Trainable params: 1,348,906
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1366 - accuracy: 0.9504 - val_loss: 0.2551 - val_accuracy: 0.9107
Epoch 5/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0973 - accuracy: 0.9637 - val_loss: 0.2846 - val_accuracy: 0.9132
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0679 - accuracy: 0.9756 - val_loss: 0.3320 - val_accuracy: 0.9140
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0509 - accuracy: 0.9816 - val_loss: 0.3405 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0358 - accuracy: 0.9869 - val_loss: 0.4642 - val_accuracy: 0.9003
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0293 - accuracy: 0.9899 - val_loss: 0.4572 - val_accuracy: 0.9073
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0246 - accuracy: 0.9916 - val_loss: 0.4756 - val_accuracy